# Sentiment Analysis Using Stanford Data

This exploration involves a large dataset of movie reviews from IMDb that has been collected by Maas et al. and used in the paper "Learning Word Vectors for Sentiment Analysis". The dataset can be downloaded from http://ai.stanford.edu/~amaas/data/sentiment/ as a zip archive.


## Importing Libraries and Processing/Loading Data

In [2]:
# Importing necessary libraries
import pyprind # For seeing the progress of loading the data
import pandas as pd
import os

In [3]:
# Converting the dataset into a dataframe object
'''
The code below will create a dataframe with a column for the text of the review and a column for the sentiment,
with 1 representing a positive sentiment and 0 representing a negative sentiment
'''

pbar = pyprind.ProgBar(50000)
labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update() # This allows us to see the action in progress!
df.columns = ['review', 'sentiment'] 

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:21


In [4]:
import numpy as np
np.random.seed(0) # Sets random seed for reproducible results
df = df.reindex(np.random.permutation(df.index)) # Shuffles the data
df.to_csv('./stanford_movie_data.csv', index=False) # for storing the data later

In [5]:
df = pd.read_csv('./stanford_movie_data.csv') #Just to confirm that the data was loaded properly
df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


## Cleaning Up The Text Data

In some cases, text data can contain unwanted characters such as HTML markup or punctuation. Since this is the case, we can clean up the text data using Python's regex library.

In [8]:
import re # regex library
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [9]:
# Next we can apply this function to each review
df['review'] = df['review'].apply(preprocessor)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.3+ KB


## Using the TF-IDF statistic and training models

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer # Used for vectorizing each review